In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from plotly.subplots import make_subplots
from numpy.core.fromnumeric import reshape
import numpy as np
import plotly.graph_objects as go

X = []; Y = []
nx = 0; ny = 0;

with open('drive/MyDrive/Work/NEU/MobileRobotics/pclX.txt') as f1:
    for line in f1.readlines():
        num = [float(i) for i in line.split(' ')]
        X.append(num)
        nx += 1;

with open('drive/MyDrive/Work/NEU/MobileRobotics/pclY.txt') as f2:
    for line in f2.readlines():
        num = [float(i) for i in line.split(' ')]
        Y.append(num)
        ny += 1;
 
X = np.transpose(np.array(X))
Y = np.transpose(np.array(Y))

original_X = go.Scatter3d(x=X[0], y=X[1], z=X[2],mode='markers',marker = dict(size = 1.5, color = 'royalblue'))
original_Y = go.Scatter3d(x=Y[0], y=Y[1], z=Y[2],mode='markers',marker = dict(size = 1.5, color = 'darkgreen'))

separate_fig = make_subplots(rows=1, cols=2,
                    specs=[[{"type": "scene"}, {"type": "scene"}]])


separate_fig.append_trace(original_X, row=1, col=1)
separate_fig.append_trace(original_Y, row=1, col=2)

separate_fig.show()
print(X.shape)

(3, 5750)


In [3]:
joint_fig = make_subplots(rows=1, cols=1,
                    specs=[[{"type": "scene"}]])

joint_fig.append_trace(original_X, row=1, col=1)
joint_fig.append_trace(original_Y, row=1, col=1)

joint_fig.show()

In [9]:
X[:,[1]]

array([[6.21000e-04],
       [6.97301e-01],
       [2.59990e-02]])

In [4]:
def EstimateCorresponences(X,Y,t,R,d_max):
    C = []
    for i in range(nx):
        Yj = Y - (np.dot(R,X[:,[i]]) + t)
        l2_norm = np.linalg.norm(Yj,axis = 0)
        argmin = np.argmin(l2_norm)
        
        if l2_norm[argmin] < d_max:
            C.append([i,argmin])
            
    #print(len(C))
    return C
    
def ComputeOptimalRigidRegistration(X,Y,C):
    K = len(C)
    x_centroid = 0; y_centroid = 0
    x_deviation = []; y_deviation = []

    for i in range(K):
        x_point = X[:,C[i][0]]; y_point = Y[:,C[i][1]]
        x_centroid += x_point
        y_centroid += y_point
        x_deviation.append(x_point)
        y_deviation.append(y_point)

    x_centroid = x_centroid/K; y_centroid = y_centroid/K
    #print(x_centroid)
    x_deviation = np.array(x_deviation - x_centroid)
    y_deviation = np.array(y_deviation - y_centroid)

    W = np.dot(np.transpose(y_deviation),x_deviation)/K
    #print(W)
    u,s,vt = np.linalg.svd(W)
    diag = np.identity(3)
    diag[2][2] = np.linalg.det(np.matmul(u,vt))
    
    R = np.matmul(u,np.matmul(diag,vt))
    t = y_centroid.reshape(3,1) - np.matmul(R,x_centroid.reshape(3,1))
    #print(t)
    #print(R)
    return t,R

def ICP(X,Y,t0,R0,d_max,num_ICP_iters):
    t = t0; R = R0
    C = []
    Xnew = X
    while num_ICP_iters!=0:
        K = 0
        print(num_ICP_iters)
        C = EstimateCorresponences(Xnew,Y,t,R,d_max)
        t,R  = ComputeOptimalRigidRegistration(Xnew,Y,C)
        num_ICP_iters -= 1

    return t,R,C



In [5]:
t,R,C = ICP(X,Y,0,np.identity(3),0.25,30)
Xnew = np.dot(R,X) + t

30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


In [ ]:
transformed_X = go.Scatter3d(x=Xnew[0], y=Xnew[1], z=Xnew[2],mode='markers',marker = dict(size = 1.5, color = 'deeppink'))

joint_new = make_subplots(rows=1, cols=1,
                    specs=[[{"type": "scene"}]])

joint_new.append_trace(original_Y, row=1, col=1)
joint_new.append_trace(transformed_X, row=1, col=1)
#joint_new.append_trace(original_X, row=1, col=1)

joint_new.show()

In [ ]:
print("t = ",t)
print("R = ",R)

t =  [[ 0.49661487]
 [-0.29392971]
 [ 0.29645004]]
R =  [[ 0.95126601 -0.15043058 -0.26919069]
 [ 0.22323628  0.9381636   0.26460276]
 [ 0.21274056 -0.31180074  0.92602471]]


In [ ]:
RMSE = 0
for i in C:
    RMSE += (np.linalg.norm(Y[:,i[1]] - Xnew[:,i[0]]))**2
RMSE = np.sqrt(RMSE/(len(C)))
print("RMSE = ",RMSE)

NameError: ignored

In [7]:
print(np.identity(3))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
